# notebook for prototyping the environement of the OLA project

In [1]:
import random

In [2]:
class Product:
    # constructor
    def __init__(self, id_):
        self.id_ = id_
        self.reservation_price = random.randint(0,100)
        self.purchase_probability = 0.5
        self.secondaries = []
    
    
    # add a product to the list of secondary product with the associated probability
    def add_secondary(self, proba, product ):
        self.secondaries.append([proba, product])

In [3]:
# the recommandation system was a bit hard to handle
# this eliviate this issue
class Recommandation:
    # constructor
    def __init__(self):
        self.secondary_1 = None
        self.secondary_2 = None
        self.proba_1 = 0
        self.proba_2 = 0
        # true if we have 2 secondaries
        self.has_2 = False
    
    
    # return true if successfully added
    def add_secondary(self, product, proba):
        if self.secondary_1 is None:
            self.secondary_1 = product
            self.proba_1 = proba
        elif self.secondary_2 is None:
            self.has_2 = True
            self.secondary_2 = product
            self.proba_2 = proba
        else:
            return False
        return True

In [4]:
class User:
    # constructor
    def __init__(self):
        # stores the id of product seen
        self.product_seen = []
        self.cart = []
        self.quantities = []
    
    
    # primary = None if first visit    
    def visit(self, e_commerce, primary = None):
        if primary is None:
            print("user", id(self), "starts visit_ at shop :", id(e_commerce))
            primary = e_commerce.draw_first_product(self)
            if random.random() < primary.purchase_probability:
                print("user", id(self), "buys prod :", primary.id_)
                self.check_product(primary)
                self.visit(e_commerce, primary)
            else:
                print("End of visit, nothing bought.")
                return
        else:
            r = e_commerce.recommand_products(self, primary)
            if r.secondary_1 is None:
                print("end of visit, no secondary from", primary.id_)
                return
            else:
                if random.random() < r.proba_1 :
                        print("user", id(self), "buys secondary 1 :", r.secondary_1.id_)
                        self.check_product(r.secondary_1)
                        self.visit(e_commerce, r.secondary_1)
                # maybe this is NOT the correct behaviour
                # we should probably test the other secondary before returning
                else:
                    print("end of visit, user didn't buy secondary 1")
                    return
                if r.has_2:
                    if random.random() < r.proba_2*e_commerce.lambda_ :
                        print("user", id(self), "buys secondary 2 :", r.secondary_2.id_)
                        self.check_product(r.secondary_2)
                        self.visit(e_commerce, r.secondary_2)
                    else: 
                        print("end of branch, the user didn't buy secondary 1 & 2")
                        return
        self.finish_visit(e_commerce)
        

    def check_product(self, product, quantity = random.randint(1,10)):
        self.product_seen.append(product.id_)
        self.add_to_cart(product, quantity)
            
            
    # add a product to the cart and start the recommandation process
    # p product
    # n quantity of product
    def add_to_cart(self, product, n):
        print("product", product.id_, "added to cart of", id(self))
        self.cart.append(product)
        self.quantities.append(n)
            
    
    # at the end of the visit, the user buy everything he has in its cart
    def finish_visit(self, e_commerce):
        total = 0
        i = 0
        for item in self.cart:
            # price * quantity
            total += item.reservation_price * self.quantities[i]
            i+=1 # lazy for loop
        print("user", id(self), "finished visit at store", id(e_commerce), "for a total spending of", total)
        e_commerce.profit(total) 

In [5]:
class E_commerce:
    # constructor
    def __init__(self):
        self.products = []
        self.users = []
        # amount of money spent by the users
        self.cash = 0
        # probability that the user check the second product
        # constant and known accros every project proposal
        self.lambda_ = 0.5 # arbitrary
    
    def set_lambda(self, new_lambda):
        self.lambda_ = new_lambda
    
    
    def add_product(self, product):
        self.products.append(product)
    
    
    # handy to add every product at once
    def set_products(self, product_list):
        self.products = product_list
    
    
    def draw_first_product(self, user):
        return random.sample(self.products, 1)[0];


    # return a list of recommandation
    def recommand_products(self, user, primary):
        r = Recommandation()
        for secondary in primary.secondaries:
            if secondary[0].id_ not in user.product_seen:
                if not r.add_secondary(secondary[0], secondary[1]):
                    break;
        return r
    
    
    def profit(self, total):
        self.cash += total
        print("shop", id(self), "now has", self.cash, "euros of profits")

In [42]:
# actual scenario ???

# create one shop 
shop = E_commerce()

# create 5 product
p1 = Product(1)
p2 = Product(2)
p3 = Product(3)
p4 = Product(4)
p5 = Product(5)

shop.set_products([p1, p2, p3, p4, p5])

# The shop doesn't have to "own" the user base 
u1 = User()



# create the relations between products
p1.add_secondary(p3, 0.2)

p2.add_secondary(p1, 0.1)
p2.add_secondary(p4, 0.3)

p3.add_secondary(p2, 0.2)
p3.add_secondary(p4, 0.1)

p4.add_secondary(p1, 0.2)


# let's run the scenario
u1.visit(shop)


user 4388922704 starts visit_ at shop : 4388923424
user 4388922704 buys prod : 4
product 4 added to cart of 4388922704
user 4388922704 buys secondary 1 : 1
product 1 added to cart of 4388922704
user 4388922704 buys secondary 1 : 3
product 3 added to cart of 4388922704
user 4388922704 buys secondary 1 : 2
product 2 added to cart of 4388922704
end of visit, no secondary from 2
user 4388922704 finished visit at store 4388923424 for a total spending of 820
shop 4388923424 now has 820 euros of profits
user 4388922704 finished visit at store 4388923424 for a total spending of 820
shop 4388923424 now has 1640 euros of profits
user 4388922704 finished visit at store 4388923424 for a total spending of 820
shop 4388923424 now has 2460 euros of profits
user 4388922704 finished visit at store 4388923424 for a total spending of 820
shop 4388923424 now has 3280 euros of profits


In [45]:
# WHACKY scenario ???

# create one shop 
shop = E_commerce()

# create 5 product
p1 = Product(1)
p2 = Product(2)
p3 = Product(3)
p4 = Product(4)
p5 = Product(5)

shop.set_products([p1, p2, p3, p4, p5])

# The shop doesn't have to "own" the user base 
u1 = User()



# create the relations between products
p1.add_secondary(p3, 1)

p2.add_secondary(p1, 1)
p2.add_secondary(p4, 1)

p3.add_secondary(p2, 1)
p3.add_secondary(p4, 1)

p4.add_secondary(p1, 1)


# let's run the scenario
u1.visit(shop)


user 4388865600 starts visit_ at shop : 4388942320
user 4388865600 buys prod : 2
product 2 added to cart of 4388865600
user 4388865600 buys secondary 1 : 1
product 1 added to cart of 4388865600
user 4388865600 buys secondary 1 : 3
product 3 added to cart of 4388865600
user 4388865600 buys secondary 1 : 4
product 4 added to cart of 4388865600
end of visit, no secondary from 4
user 4388865600 finished visit at store 4388942320 for a total spending of 1240
shop 4388942320 now has 1240 euros of profits
user 4388865600 finished visit at store 4388942320 for a total spending of 1240
shop 4388942320 now has 2480 euros of profits
user 4388865600 buys secondary 2 : 4
product 4 added to cart of 4388865600
end of visit, no secondary from 4
user 4388865600 finished visit at store 4388942320 for a total spending of 1360
shop 4388942320 now has 3840 euros of profits
user 4388865600 finished visit at store 4388942320 for a total spending of 1360
shop 4388942320 now has 5200 euros of profits


In [8]:
id(None)

4341234312